In [7]:
%matplotlib ipympl
from skimage import io as skio
import matplotlib.pyplot as plt
url = 'batch/France/105.jpg'
# url = 'batch/A_curious_herbal/2.jpg'
img = skio.imread(url)

In [8]:
plt.figure()
skio.imshow(img)
print("shape of image: {}".format(img.shape))
print("dtype of image: {}".format(img.dtype))

A Jupyter Widget

shape of image: (760, 543, 3)
dtype of image: uint8


In [9]:
from skimage import color
grey = color.rgb2grey(img)
plt.figure()
skio.imshow(grey,cmap=plt.cm.gray)

A Jupyter Widget

In [10]:
from skimage import filters
sobel = filters.sobel(grey)
plt.figure()
skio.imshow(sobel)

A Jupyter Widget

In [12]:
# skio.imsave("sobel.png",sobel,cmap=plt.cm.gray)

In [13]:
from skimage import feature
edges1 = feature.canny(grey)
edges2 = feature.canny(grey, sigma=2)
plt.figure()
skio.imshow(edges1)
plt.figure()
skio.imshow(edges2)
# plt.imsave("t.jpg",edges2,cmap=plt.cm.gray_r)

A Jupyter Widget

A Jupyter Widget

In [15]:
import numpy as np
markers = np.zeros_like(grey)
markers[grey < 0.6] = 2 #黑色部分
markers[grey > 0.9] = 1 #白色部分
plt.figure()
skio.imshow(markers)

A Jupyter Widget

C:\Users\yeshu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:77: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warn("Float image out of standard range; displaying "


In [16]:
from skimage.morphology import watershed
segmentation = watershed(sobel, markers)
plt.figure()
skio.imshow(segmentation-1)
# segmentation-1

A Jupyter Widget

C:\Users\yeshu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_plugins\matplotlib_plugin.py:74: UserWarning: Low image data range; displaying image with stretched contrast.
  warn("Low image data range; displaying image with "


In [17]:
import numpy as np
img2 = np.dstack((img,(segmentation-1).astype(np.uint8)*255))
plt.figure()
skio.imshow(img2)
# skio.imsave("t2223.png",img2,)
# img2

A Jupyter Widget

In [18]:
from scipy import ndimage as ndi
from skimage import measure
label_objects = measure.label(segmentation-1)
sizes = np.bincount(label_objects.ravel())
sizes[0]=0
mask_sizes = sizes == max(sizes)
# np.shape(label_objects)
# plt.figure()
(mask_sizes[label_objects]).astype(np.uint8)
img2 = np.dstack((img,(mask_sizes[label_objects]).astype(np.uint8)*255))
plt.figure()
skio.imshow(img2)

A Jupyter Widget

In [20]:
def bbox1(img):
    a = np.where(img != 0)
    bbox = np.min(a[0]), np.max(a[0]), np.min(a[1]), np.max(a[1])
    return bbox

In [21]:
a,b,c,d=bbox1((mask_sizes[label_objects]).astype(np.uint8))


In [22]:
plt.figure()
skio.imshow(img2[a:b+1,c:d+1])

A Jupyter Widget